In [1]:
import os
import torch.backends.cudnn as cudnn
import torch
import yaml
from utils import AttrDict
import pandas as pd
from train import train
from data_prepare import prepare_data

#### Install packages

In [2]:
# !pip install torch
# !pip install natsort
# !pip install torchvision
# !pip install pyyaml
# !pip install pandas
# !pip install nltk

In [3]:
cudnn.benchmark = True
cudnn.deterministic = False
torch.cuda.is_available()

True

#### Training

In [4]:
# image_path = raw_data_path = os.path.join('raw_data','train_images_30k')
# prepare_data(image_path)

In [5]:
def get_config(file_path):
    with open(file_path, 'r', encoding="utf8") as stream:
        opt = yaml.safe_load(stream)
    opt = AttrDict(opt)
    if opt.lang_char == 'None':
        pass
        # characters = ''
        # for data in opt['select_data'].split('-'):
        #     csv_path = os.path.join(opt['train_data'], 'training' ,data, 'labels.csv')
        #     df = pd.read_csv(csv_path, sep='^([^,]+),', engine='python', usecols=['filename', 'words'], keep_default_na=False)
        #     all_char = ''.join(df['words'])
        #     characters += ''.join(set(all_char))
        # characters = sorted(set(characters))
        # opt.character= ''.join(characters)
        # print('char: ',opt.character)
    else:
        characters = ''
        for data in opt['select_data'].split('-'):
            csv_path = os.path.join(opt['train_data'], 'training' ,data, 'labels.csv')
            df = pd.read_csv(csv_path, sep='^([^,]+),', engine='python', usecols=['filename', 'words'], keep_default_na=False)
            all_char = ''.join(df['words'])
            characters += ''.join(set(all_char))
        characters = sorted(set(characters))
        opt.character= ''.join(characters)
        print(f'experiment: {opt.experiment_name}')
        print(f'char in train_set:{opt.character} class{len(opt.character)}')
        opt.character = opt.lang_char
        print('char in config:',opt.character)
    os.makedirs(f'./saved_models/{opt.experiment_name}', exist_ok=True)
    return opt

albe to config in config_files/*.yaml
check list: \
&emsp;lang_char \
&emsp;experiment_name \
&emsp;num_iter \
&emsp;saved_model \
&emsp;lr \
&emsp;model architecture

In [6]:
opt = get_config("config_files/crnn_30k.yaml")
train(opt, amp=False)

experiment: crnn_30k_lr_25e-2_test
char in train_set:  !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz|~กขฃคฅฆงจฉชซฌญฎฏฐฑฒณดตถทธนบปผฝพฟภมยรฤลฦวศษสหฬอฮฯะัาำิีึืุูเแโใไๅๆ็่้๊๋์ํ๐๑๒๓๔๕๖๗๘๙–‘’
char in config: 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`|~กขฃคฅฆงจฉชซฌญฎฏฐฑฒณดตถทธนบปผฝพฟภมยรฤลฦวศษสหฬอฮฯะัาำิีึืุูเแโใไๅๆ็่้๊๋์ํ๐๑๒๓๔๕๖๗๘๙–‘ ’
--------------------------------------------------------------------------------
dataset_root: all_data/training
opt.select_data: ['']
opt.batch_ratio: ['1']
--------------------------------------------------------------------------------
dataset_log dataset_root:    all_data/training	 dataset: 
all_data/training/training
sub-directory:	/training	 num samples: 27000
num total samples of : 27000 x 1.0 (total_data_usage_ratio) = 27000
num samples of  per batch: 32 x 1.0 (batch_ratio) = 32
--------------------------------------------------------------------------------
T

  1%|          | 6/1000 [00:08<11:55,  1.39it/s]  

KeyboardInterrupt: 

In [ ]:
label_path = [ './saved_models/crnn_30k_lr_5e-1/log_loss_train.txt',
]
loss_list = []
for path in label_path:
    with open(path, encoding='utf8') as file:
        lines = file.readlines()
        for line in lines:
            if not line.startswith('experiment_name'):
                loss_list.append(float(line.split(' ')[-2]))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
print(len(loss_list))
batch_size = 32
iteration = len(loss_list)*10
epoch = (iteration*batch_size) // 26859
epoch_avg = np.array([])
for i in range(epoch):
    epoch_avg = np.append(epoch_avg, np.mean(loss_list[int(i*26859/320):(i+1)*int(i*26859/320)]))
plt.figure(figsize=(10, 10), dpi=80)
start_row = 0
start = int(start_row /10)
# plt.plot(np.arange(len(loss_list[start:]))*10+start_row, loss_list[start:])
plt.plot(epoch_avg[:])
# plt.title(f'Training loss afer {start_row} iterations')
plt.title(f'Training loss')
plt.xlabel('iterations')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()